<a href="https://colab.research.google.com/github/Vasanthnaik11/sustainability-internship-project/blob/main/solar_pannel_damage_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ============================
# 📦 STEP 1 — Setup Environment
# ============================
from google.colab import drive
drive.mount('/content/drive')  # Optional: if you want to save files to Drive

import os
import zipfile
import shutil
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# ============================
# 📁 STEP 2 — Unzip Dataset
# ============================
zip_path = "/content/archive (2).zip"   # uploaded dataset path - Make sure this path is correct!
extract_path = "/content/dataset"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("✅ Dataset extracted successfully to:", extract_path)

# View folder structure
for root, dirs, files in os.walk(extract_path):
    print(root, len(files), "files")

# ============================
# 🧹 STEP 3 — Organize Folder
# ============================
# (Skip this step if your dataset already has folders like 'healthy', 'dusty', 'damaged')

# Example (uncomment if needed):
# os.makedirs("/content/dataset/healthy", exist_ok=True)
# os.makedirs("/content/dataset/dusty", exist_ok=True)
# os.makedirs("/content/dataset/damaged", exist_ok=True)
# Move or rename folders as required

# ============================
# 🧠 STEP 4 — Data Generators
# ============================
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,  # 80% train / 20% val
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

train_generator = train_datagen.flow_from_directory(
    extract_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    extract_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

print("✅ Data preprocessing complete.")

# ============================
# 🖼️ STEP 5 — Visualize Samples
# ============================
x_batch, y_batch = next(train_generator)
plt.figure(figsize=(8,8))
for i in range(9):
    plt.subplot(3,3,i+1)
    plt.imshow(x_batch[i])
    plt.axis('off')
plt.suptitle("Sample Augmented Images", fontsize=14)
plt.show()

Mounted at /content/drive
✅ Dataset extracted successfully to: /content/dataset


FileNotFoundError: [Errno 2] No such file or directory: '/content/dataset'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

file_path = "/content/archive (2).zip"

if os.path.exists(file_path):
  print(f"The file {file_path} exists.")
else:
  print(f"The file {file_path} does not exist.")

In [2]:
# ============================================================
# 🌞 Solar Panel Damage Detection using Deep Learning (CNN)
# ============================================================
# Developed by: Vasanth Naik
# Environment: Google Colab / Jupyter Notebook
# ============================================================

import os
import zipfile
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from google.colab import files


In [3]:
# Upload your dataset zip file (should contain train/test folders or images)
uploaded = files.upload()
zip_file = list(uploaded.keys())[0]

# Extract files
dataset_path = "/content/dataset"
os.makedirs(dataset_path, exist_ok=True)

with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall(dataset_path)

print("✅ Dataset extracted successfully at:", dataset_path)


Saving archive (3).zip to archive (3) (1).zip
✅ Dataset extracted successfully at: /content/dataset


In [4]:
# Image data generator for training and validation
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=15,
    zoom_range=0.1,
    horizontal_flip=True
)

IMG_SIZE = (128, 128)
BATCH_SIZE = 32

train_gen = datagen.flow_from_directory(
    dataset_path,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

val_gen = datagen.flow_from_directory(
    dataset_path,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

print("✅ Classes detected:", train_gen.class_indices)


Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.
✅ Classes detected: {}


In [5]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(128,128,3)),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(len(train_gen.class_indices), activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 0)              │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,304,640 (12.61 MB)

 Trainable params: 3,304,640 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
EPOCHS = 0

history = model.fit(
    train_gen,
    epochs=EPOCHS,
    validation_data=val_gen
)


In [ ]:
plt.figure(figsize=(12,4))

plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.title("Model Accuracy")

plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.title("Model Loss")

plt.show()


In [13]:
model.save("/content/solar_panel_damage_cnn.h5")
print("✅ Model saved as solar_panel_damage_cnn.h5")

files.download("/content/solar_panel_damage_cnn.h5")


✅ Model saved as solar_panel_damage_cnn.h5


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [14]:
from tensorflow.keras.preprocessing import image

def predict_image(img_path):
    img = image.load_img(img_path, target_size=(128, 128))
    img_array = image.img_to_array(img)/255.0
    img_array = np.expand_dims(img_array, axis=0)
    prediction = model.predict(img_array)
    class_idx = np.argmax(prediction)
    label = list(train_gen.class_indices.keys())[class_idx]
    print(f"🟩 Predicted class: {label}")

# Example:
# predict_image("/content/dataset/Damaged/image_1.jpg")


In [ ]:
# ============================================================
# 🌞 Solar Panel Damage Detection using Deep Learning (CNN)
# Streamlit Web Application
# Developed by: Vasanth Naik
# ============================================================

import streamlit as st
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np
from PIL import Image

# ------------------------------------------------------------
# Page Configuration
# ------------------------------------------------------------
st.set_page_config(
    page_title="Solar Panel Damage Detection",
    page_icon="🔆",
    layout="centered"
)

st.title("🌞 Solar Panel Damage Detection using CNN")
st.markdown("Upload a solar panel image to detect whether it's **Healthy**, **Dusty**, or **Damaged**.")

# ------------------------------------------------------------
# Load Trained Model
# ------------------------------------------------------------
@st.cache_resource
def load_model():
    model = tf.keras.models.load_model("solar_panel_damage_cnn.h5")
    return model

model = load_model()

# Define your class labels (adjust based on your dataset)
CLASS_NAMES = ['Damaged', 'Dusty', 'Healthy']

# ------------------------------------------------------------
# Image Upload Section
# ------------------------------------------------------------
uploaded_file = st.file_uploader("📸 Upload a Solar Panel Image", type=["jpg", "jpeg", "png"])

if uploaded_file is not None:
    # Display uploaded image
    img = Image.open(uploaded_file)
    st.image(img, caption="Uploaded Image", use_column_width=True)

    # Preprocess image
    img = img.resize((128, 128))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # Predict
    prediction = model.predict(img_array)
    class_index = np.argmax(prediction)
    result = CLASS_NAMES[class_index]
    confidence = np.max(prediction) * 100

    # Display result
    st.success(f"✅ Predicted: **{result}**")
    st.info(f"Confidence: {confidence:.2f}%")

# ------------------------------------------------------------
# Footer
# ------------------------------------------------------------
st.markdown("---")
st.markdown("Developed with ❤️ by **Vasanth Naik** | Powered by TensorFlow & Streamlit")


In [16]:
# Install streamlit
!pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 80.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 123.5 MB/s eta 0:00:00


In [ ]:
streamlit run app.py


In [ ]:
!streamlit run app.py




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.185.236.231:8501



In [18]:
# Save the Streamlit app code to a file
app_code = """
# ============================================================
# 🌞 Solar Panel Damage Detection using Deep Learning (CNN)
# Streamlit Web Application
# Developed by: Vasanth Naik
# ============================================================

import streamlit as st
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np
from PIL import Image

# ------------------------------------------------------------
# Page Configuration
# ------------------------------------------------------------
st.set_page_config(
    page_title="Solar Panel Damage Detection",
    page_icon="🔆",
    layout="centered"
)

st.title("🌞 Solar Panel Damage Detection using CNN")
st.markdown("Upload a solar panel image to detect whether it's **Healthy**, **Dusty**, or **Damaged**.")

# ------------------------------------------------------------
# Load Trained Model
# ------------------------------------------------------------
@st.cache_resource
def load_model():
    model = tf.keras.models.load_model("solar_panel_damage_cnn.h5")
    return model

model = load_model()

# Define your class labels (adjust based on your dataset)
# These class names should match the order of classes found by flow_from_directory
# You can find this order in train_gen.class_indices after running the data generator cell.
# Example: CLASS_NAMES = ['Damaged', 'Dusty', 'Healthy']
# Based on the error message in u4q2pQ-YFE3V, the classes were not detected.
# You will need to replace this with the actual class names once the data generators work.
CLASS_NAMES = ['Class1', 'Class2', 'Class3'] # Replace with actual class names

# ------------------------------------------------------------
# Image Upload Section
# ------------------------------------------------------------
uploaded_file = st.file_uploader("📸 Upload a Solar Panel Image", type=["jpg", "jpeg", "png"])

if uploaded_file is not None:
    # Display uploaded image
    img = Image.open(uploaded_file)
    st.image(img, caption="Uploaded Image", use_column_width=True)

    # Preprocess image
    img = img.resize((128, 128))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # Predict
    prediction = model.predict(img_array)
    class_index = np.argmax(prediction)
    result = CLASS_NAMES[class_index]
    confidence = np.max(prediction) * 100

    # Display result
    st.success(f"✅ Predicted: **{result}**")
    st.info(f"Confidence: {confidence:.2f}%")

# ------------------------------------------------------------
# Footer
# ------------------------------------------------------------
st.markdown("---")
st.markdown("Developed with ❤️ by **Vasanth Naik** | Powered by TensorFlow & Streamlit")
"""

with open("app.py", "w") as f:
    f.write(app_code)

print("✅ app.py created successfully")

✅ app.py created successfully
